# Daily Stopwatch Data Science: KAThai Visualization

Note: this is a part of visualizing Khan Acadmy Thailand videos information. For more information, see https://khanacademythailand.org

## Stage 1: Ask a question

My objective is to create a visualization tool for Khan Academy Thailand videos.

There is no performance measure for the visualization yet.

## Stage 2: Set the environment up and get data

First, set up a directory for data and link it to this workplace. Download data into your choice of directory.

**Note:** in order to work properly, you need to set up a terminal with UTF-8 format. See more [here](http://stackoverflow.com/questions/10561923/unicodedecodeerror-ascii-codec-cant-decode-byte-0xef-in-position-1).

In [15]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [16]:
#Set up the environment
import pandas as pd                        #Pandas
import numpy as np                         #Numpy

#Reading directly from Google sheets is complicated. Let's do a simple version by going to a local CSV file. 
#For more information about Google Sheets API, see https://developers.google.com/sheets/api/quickstart/python
'''
from StringIO import StringIO              #Properly read goodle sheets got moved to io in python3.

import requests

#read Google sheets data
r = requests.get('https://docs.google.com/spreadsheets/d/1uJSAtyUTQX9pmbI8YlXg_3Z15vKy3Bu4-wvzmoYOep0&output=csv')
data = r.content

df = pd.read_csv(StringIO(data))#, index_col=0)
'''

# Set up data directory
DataDir = "C:/Users/Admin/Documents/data/"
filename = "ThaiKAVideoProcess.xlsx"

df = pd.pandas.read_excel(DataDir+filename,sheetname='AllAvailableThaiVideos')

In [17]:
df[:10]

Domain  \
0   Math   
1   Math   
2   Math   
3   Math   
4   Math   
5   Math   
6   Math   
7   Math   
8   Math   
9   Math   

                                                              DomainThai  \
0                                      ??????????...                       
1                                      ??????????...                       
2                                      ??????????...                       
3                                      ??????????...                       
4                                      ??????????...                       
5                                      ??????????...                       
6                                      ??????????...                       
7                                      ??????????...                       
8                                      ??????????...                       
9                                      ??????????...                       

      Subject  \
0  Early math   
1  Early math   
2  Early math   
3  Early math   
4  Early math   
5  Early math   
6  Early math   
7  Early math   
8  Early math   
9  Early math   

                                                             SubjectThai  \
0                                      ??????????...                       
1                                      ??????????...                       
2                                      ??????????...                       
3                                      ??????????...                       
4                                      ??????????...                       
5                                      ??????????...                       
6                                      ??????????...                       
7                                      ??????????...                       
8                                      ??????????...                       
9                                      ??????????...                       

      Topic                                   TopicThai  \
0  Counting                          ??????               
1  Counting                          ??????               
2  Counting                          ??????               
3  Counting                          ??????               
4  Counting                          ??????               
5  Counting                          ??????               
6  Counting                          ??????               
7  Counting                          ??????               
8  Counting                          ??????               
9  Counting                          ??????               

                  Tutorial  \
0   Counting small numbers   
1   Counting small numbers   
2         Numbers 0 to 120   
3         Numbers 0 to 120   
4         Counting objects   
5         Counting objects   
6         Counting objects   
7  Comparing small numbers   
8  Comparing small numbers   
9  Comparing small numbers   

                                                                                        TutorialThai  \
0                                   ?????????????...                                                   
1                                   ?????????????...                                                   
2                                    ????? 0 ??? ...                                                   
3                                    ????? 0 ??? ...                                                   
4                                        ????????...                                                   
5                                        ????????...                                                   
6                                        ????????...                                                   
7                        ????????????????????????...                                                   
8                        ????????????????????????...                                      

We will use an interactive visualization tool called Bokeh. See an [overview](http://nbviewer.jupyter.org/github/bokeh/bokeh-notebooks/blob/master/quickstart/quickstart.ipynb) and [how to install](http://bokeh.pydata.org/en/0.11.0/docs/installation.html).

In [18]:
from bokeh.models import ColumnDataSource, HoverTool, OpenURL, TapTool
from bokeh.plotting import figure, show
from bokeh.sampledata.periodic_table import elements
from bokeh.io import vplot

from bokeh.layouts import row, widgetbox
from bokeh.models import Select
from bokeh.palettes import Spectral5
from bokeh.plotting import curdoc, figure
from bokeh.sampledata.autompg import autompg

In [19]:
#make output appeared in notebook
from bokeh.plotting import output_notebook
output_notebook()

Loading BokehJS ...

## Stage 3: Explore the data

In [20]:
d_complete = df[(df.IsComplete==1)|(df.IsComplete==0)]
d_complete.reset_index(inplace=True)

Here, we want to assign color based on the attributed called **ThaiCurriculumMapping**.

In [21]:
DomainList = list(d_complete.Domain.unique())

In [22]:
MappingList = list(d_complete.ThaiCurriculumMapping.unique())
#l = pd.DataFrame(MappingList).sort()
#l
d_complete['Color'] = "0"
for i in range(0,len(d_complete)):
    for j in range(0,len(MappingList)):
        if d_complete.ThaiCurriculumMapping[i] == MappingList[j]:
            d_complete.loc[i,'Color'] = str(j)    

colormap = {
    "0"         : "#a6cee3",
    "1"         : "#1f78b4",
    "2"         : "#fdbf6f",
    "3"         : "#b2df8a",
    "4"         : "#33a02c",
    "5"         : "#a6cee3",
    "6"         : "#1f78b4",
    "7"         : "#fdbf6f",
    "8"         : "#b2df8a",
    "9"         : "#33a02c",
    "10"        : "#bbbb88",
    "11"        : "#baa2a6",
    "12"        : "#e08e79",
}

Now we want to assign group number to each row. According to the order, we know that the rows under the same topic/subject are next to each other. So we will assign number using loop. 

In [23]:
d_complete['GroupNumber'] = 1
num = 1
for i in range(1,len(d_complete)):
    if (d_complete.Topic[i] == d_complete.Topic[i-1])&(d_complete.Subject[i] == d_complete.Subject[i-1]):
        num = num + 1
    else:
        num = 1
    d_complete.loc[i,'GroupNumber'] = num    
MaxGroupNumber = np.max(d_complete['GroupNumber'])

group_range = [str(x) for x in range(1, MaxGroupNumber + 1)] #This is equivalent to Group number in Periodic Table (columns)

Next, let's detect the smaller category called Subject.

In [24]:
# fill in Youtube link (redirect to translation page if needed)
for i in d_complete.index:
    if d_complete.loc[i,'IsComplete']==0:
        d_complete.loc[i,'Subbed Youtube Link'] = "https://www.youtube.com/timedtext_video?v=" + d_complete.loc[i,'Original YoutubeID']
            

In [25]:
def create_figure():
    df = d_complete[d_complete.Domain==domain.value]
    df.reset_index(inplace=True)
    SubjectList = list(df.Subject.unique())
    SubplotList = [0]*len(SubjectList)    #Create subplots for different subjects
    for i in range(0,len(SubjectList)):
        df_small = df[df['Subject']==SubjectList[i]]
        #Here we will associate each row with each topic. This is equivalent to Roman Letter in Periodic Table (rows)
        TopicList = list(df_small.Topic.unique()) 
        if option.value == 'KA link':
            source = ColumnDataSource(
                data=dict(
                    GroupNumber= df_small['GroupNumber'],
                    Topic = [str(x) for x in df_small['Topic']],
                    Tutorial = df_small['Tutorial'],
                    TutorialThai = df_small['TutorialThai'],
                    KASiteName = df_small['KASiteName'],
                    KASiteNameThai = df_small['KASiteNameThai'],
                    SiteLink = df_small['KA site link'],
                    type_color= [colormap[x] for x in df_small['Color']],
                    alpha = [0.7]*len(df_small),
                )
            )
        if option.value == 'sub':
            source = ColumnDataSource(
                data=dict(
                    GroupNumber= df_small['GroupNumber'],
                    Topic = [str(x) for x in df_small['Topic']],
                    Tutorial = df_small['Tutorial'],
                    TutorialThai = df_small['TutorialThai'],
                    KASiteName = df_small['KASiteName'],
                    KASiteNameThai = df_small['KASiteNameThai'],
                    SiteLink = df_small['Subbed Youtube Link'],
                    type_color= [colormap[x] for x in df_small['Color']],
                    alpha = [0.1 + 0.6*x for x in df_small['IsComplete']],
                )
            )

        p = figure(title= SubjectList[i], tools="tap,hover",
                   x_range=group_range, y_range=list(reversed(TopicList)), y_axis_label="Topic")
        p.plot_width = 1800
        p.plot_height = 24*(len(TopicList)+2)
        p.toolbar_location = None
        p.outline_line_color = None

        p.rect("GroupNumber", "Topic", 0.9, 0.9, source=source,
               fill_alpha="alpha", color="type_color")

        p.grid.grid_line_color = None

        #Hover to get information
        p.select_one(HoverTool).tooltips=[
                    ("Tutorial","@Tutorial"),
                    ("TutorialThai","@TutorialThai"),
                    ("KASiteName","@KASiteName"),
                    ("KASiteNameThai","@KASiteNameThai"),
                ]

        #Click to go to the website
        url = "@SiteLink"
        taptool = p.select(type=TapTool)
        taptool.callback = OpenURL(url=url)

        SubplotList[i] = p
    # This is suppose to be combined plot    
    n = len(SubjectList)
    l = "p=vplot("
    c = ["SubplotList["+str(i)+"]," for i in range(0,n)]
    for i in range(0,n):
        l = l + c[i]
    l = l + ")"
    exec(l)
    
    return p

In [26]:
def update(attr, old, new):
    layout.children[1] = create_figure()

#Direct to different subjects
domain = Select(title='Domain', value='Computing', options=DomainList)
domain.on_change('value', update)

#Redirect to Khan website or Videos
option = Select(title='Option', value='KA link', options=['KA link','sub'])
option.on_change('value', update)

controls = widgetbox([domain,option], width=200)
#controls = widgetbox([domain], width=200)
layout = row(controls, create_figure())

curdoc().add_root(layout)
curdoc().title = "Khan Academy Thailand"

You need to connect with Bokeh server by typing 'bokeh serve' in a command prompt first. See [here](  http://bokeh.pydata.org/en/0.11.0/docs/user_guide/server.html#userguide-server-output-server) for more information.

In [27]:
from bokeh.client import push_session
# open a session to keep our local document in sync with server
session = push_session(curdoc())

In [28]:
session.show() # open the document in a browser
session.loop_until_closed() # run forever

Here are some ideas to develop this further:

1. Once clicked, give dialog box instead of just jump to URL directly (so can go to sub/dub/KA link).
2. Create channels to translators (video incomplete, please help). Use 'fill_alpha' to create such effect.
3. Do something to distinquish different tutorials under the same topic.

*Note to myself:* I spent at least 5 Pomodoros (or more) to figure out interactive parts. Right now it is still buggy (due to data in the domain called Science), otherwise OK. The larger problem is that we want to stay away from Bokeh server as it has Unicode/ UTF-8 issue when use Application technique. We want to avoid it altogether. Not sure how to do it yet. Let's do a better version next round. 